01_analise_exploratoria – Análise Exploratória de Dados (EDA)
Neste notebook são realizadas as etapas de:
- Carregamento e limpeza dos dados brutos;
- Tratamento de valores ausentes e padronização de variáveis;
- Análise descritiva;
- Visualizações iniciais para identificar tendências, correlações e outliers.

O objetivo é compreender o comportamento dos dados e definir hipóteses para a modelagem.

In [79]:
# Bibliotecas
import pandas as pd
import numpy as np
import plotly.express as plt


In [80]:
# Definindo o path base
path = 'C:/Users/joao9/Documents/Area_de_Trabalho_Python/Portfolio/Portifolio_Joao_Guilherme_Chimanski/'

In [81]:
# Leitura dos dados
df_IPCA = pd.read_csv(path + 'data/raw/IPCA.csv', sep='\t')
df_CEPEA = pd.read_csv(path + 'data/raw/CEPEA_ESALQ.csv', sep='\t')

In [82]:
df_IPCA.head()


,Unnamed: 0,julho 1994,agosto 1994,setembro 1994,outubro 1994,novembro 1994,dezembro 1994,janeiro 1995,fevereiro 1995,março 1995,...,fevereiro 2025,março 2025,abril 2025,maio 2025,junho 2025,julho 2025,agosto 2025,setembro 2025,Unnamed: 376,Unnamed: 377
0,Brasil,"6,84","1,86","1,53","2,62","2,81","1,71","1,7","1,02","1,55",...,"1,31","0,56","0,43","0,26","0,24","0,26","-0,11","0,48",NaN,NaN


In [83]:
df_CEPEA.head()


,Data,À vista R$,À vista US$
0,13/03/2006,"27,66","12,96"
1,14/03/2006,"27,45","12,91"
2,15/03/2006,"27,58",13
3,16/03/2006,"26,97","12,78"
4,17/03/2006,"26,96","12,68"


In [84]:
df_IPCA = df_IPCA.transpose()
#linha como variavel
df_IPCA.columns = df_IPCA.iloc[0]
df_IPCA = df_IPCA.reset_index().rename(columns={'index':'orig_index'})
df_IPCA

Unnamed: 0,orig_index,Brasil
0,Unnamed: 0,Brasil
1,julho 1994,"6,84"
2,agosto 1994,"1,86"
3,setembro 1994,"1,53"
4,outubro 1994,"2,62"
...,...,...
373,julho 2025,"0,26"
374,agosto 2025,"-0,11"
375,setembro 2025,"0,48"
376,Unnamed: 376,NaN


In [85]:
# Ajuste do formato da data no DataFrame do IPCA
muda_formato_data = {
    'janeiro': '01', 'fevereiro': '02', 'março': '03', 
    'abril': '04', 'maio': '05', 'junho': '06',
    'julho': '07', 'agosto': '08', 'setembro': '09', 
    'outubro': '10', 'novembro': '11', 'dezembro': '12'
}

regex = r'(\w+)(?:\s+de)?\s+(\d{4})'  # Regex para capturar "mês de ano" ou "mês ano"

coluna = df_IPCA.columns[0]

df_IPCA[coluna] = df_IPCA[coluna].astype(str).str.lower().str.replace(
    regex,
    lambda m: f"{muda_formato_data[m.group(1)]}/{m.group(2)}",
    regex=True
)
# remover a primeira linha que virou header
df_IPCA = df_IPCA.drop(0).reset_index(drop=True)
#substitui virgula por ponto e converte para numerico
for col in df_IPCA.columns[1:]:
    df_IPCA[col] = pd.to_numeric(df_IPCA[col].str.replace(',', '.'), errors='coerce')
df_IPCA.head()



Unnamed: 0,orig_index,Brasil
0,07/1994,6.84
1,08/1994,1.86
2,09/1994,1.53
3,10/1994,2.62
4,11/1994,2.81


In [86]:
""" # Corrige formatação e converte as duas colunas para número
for col in ['À vista R$', 'À vista US$']:
    if col in df_CEPEA.columns:
        df_CEPEA[col] = (
            df_CEPEA[col]
            .astype(str)
            .str.replace('.', '', regex=False)
            .str.replace(',', '.', regex=False)
            .astype(float)
        )
df_CEPEA['Data'] = pd.to_datetime(df_CEPEA['Data'], errors='coerce', dayfirst=True)
df_CEPEA['AnoMes'] = df_CEPEA['Data'].dt.strftime('%m/%Y')

# Calcula médias mensais em reais e dólares
colunas_validas = [c for c in ['À vista R$', 'À vista US$'] if c in df_CEPEA.columns]
df_soja_mensal = df_CEPEA.groupby('AnoMes')[colunas_validas].mean().reset_index()

df_soja_mensal.head()
 """
# troca virgula por ponto e converte para numerico
for col in ['À vista R$', 'À vista US$']:
    df_CEPEA[col] = pd.to_numeric(df_CEPEA[col].astype(str).str.replace(',', '.'), errors='coerce')
df_soja_mensal = df_CEPEA.copy()
df_soja_mensal.head()

,Data,À vista R$,À vista US$
0,13/03/2006,27.66,12.96
1,14/03/2006,27.45,12.91
2,15/03/2006,27.58,13.00
3,16/03/2006,26.97,12.78
4,17/03/2006,26.96,12.68


In [89]:
# merge dos dois dataframes pelo campo AnoMes
df_merged = pd.merge(df_soja_mensal, df_IPCA, left_on='Data', right_on='orig_index', how='join')
df_merged = df_merged.drop(columns=['orig_index'])
df_merged = df_merged.rename(columns={'À vista R$': 'Preco_Soja_real',
                                      'À vista US$': 'Preco_Soja_Dolar', 
                                      'Brasil': 'IPCA_Mensal_prc'})
df_merged.head()

KeyError: 'join'

Dados tratados, agora serão descritos e observados

In [ ]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235 entries, 0 to 234
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   AnoMes            235 non-null    object 
 1   Preco_Soja_real   235 non-null    float64
 2   Preco_Soja_Dolar  235 non-null    float64
 3   IPCA_Mensal_prc   234 non-null    float64
dtypes: float64(3), object(1)
memory usage: 7.5+ KB


In [ ]:
df_merged.describe()

,Preco_Soja_real,Preco_Soja_Dolar,IPCA_Mensal_prc
count,235.000000,235.000000,234.000000
mean,87.439276,25.886738,0.452692
std,45.909207,5.877212,0.335750
min,27.020556,12.703333,-0.680000
25%,49.571136,21.764566,0.242500
50%,74.527727,24.899130,0.430000
75%,126.863182,30.273478,0.610000
max,199.602273,42.827895,1.620000
